In [1]:
from selenium import webdriver as wd
from bs4 import BeautifulSoup as bs
import requests
import csv
import datetime

url = 'https://www.law360.com/'

driver = wd.Chrome('./chromedriver')
driver.implicitly_wait(1)
driver.get(url)
driver.implicitly_wait(1)

signin = driver.find_element_by_xpath('//*[@id="navigation_new"]/div[2]/div[1]/span/a')
signin.click()
driver.implicitly_wait(1)

In [2]:
email = driver.find_element_by_xpath('//*[@id="email"]')
pw = driver.find_element_by_xpath('//*[@id="password"]')
submit = driver.find_element_by_xpath('//*[@id="btnSignIn"]')

email.send_keys('david_hahm@fronteo.com')
pw.send_keys('fronteo1')
submit.click()
driver.implicitly_wait(1)

In [5]:
srchInput = driver.find_element_by_xpath('//*[@id="global_search_field"]')
srchBtn = driver.find_element_by_xpath('//*[@id="btnSearchResults"]')

srchInput.send_keys('Samsung Electronics v. ELM3DS')
srchBtn.click()

CasesTab = driver.find_element_by_xpath('//*[@id="results"]/ul/li[2]/a')
CasesTab.click()

